# M1M3 position data
Craig Lage - 16-Apr-23 \
Comparing different position measurements.

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from scipy.interpolate import UnivariateSpline
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState, getSlewsFromEventList
from lsst.summit.utils.efdUtils import (makeEfdClient,
                                        getEfdData,
                                        getMostRecentRowWithDataBefore,
                                        efdTimestampToAstropy,
                                       )

In [ ]:
FATABLE = M1M3FATable.FATABLE
client = makeEfdClient()
dayObs = 20230802
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(dayObs)
slews = [e for e in events if e.type==TMAState.SLEWING]
tracks = [e for e in events if e.type==TMAState.TRACKING]
print(f'Found {len(slews)} slews and {len(tracks)} tracks')

In [ ]:
for i,slew in enumerate(slews):
    start = slew.begin
    end = slew.end
    az = await client.select_time_series("lsst.sal.MTMount.azimuth","actualPosition", start, end)
    el = await client.select_time_series("lsst.sal.MTMount.elevation","actualPosition", start, end)
    try:
        azVals = az.values
        elVals = el.values
        dAz = azVals[-1] - azVals[0]
        dEl = elVals[-1] - elVals[0]
        print(i, dAz, dEl)
    except:
        continue

    

In [ ]:
slews[318].blockInfo

In [ ]:
data = {}

Gstart = Time('2023-08-02T12:00:00', scale='utc')
Gend = Time('2023-08-03T12:00:00', scale='utc')

for dayObs in [20230802]:
    els = []
    azs = []
    times = []
    xPos = []
    yPos = []
    zPos = []
    xRot = []
    yRot = []
    zRot = []

    events = eventMaker.getEvents(dayObs)
    slews = [e for e in events if e.type==TMAState.SLEWING]
    print(f'{dayObs}:found {len(slews)} slews ')
    if len(slews) < 1:
        continue
    t0 = slews[0].begin.unix_tai
    for slew in slews:
        if not (slew.begin.unix_tai > Gstart.unix_tai and slew.begin.unix_tai < Gend.unix_tai \
        and slew.end.unix_tai > Gstart.unix_tai and slew.end.unix_tai < Gend.unix_tai):
            continue

        dT = 5.0
        start = slew.end-TimeDelta(dT, format='sec')
        end = slew.end+TimeDelta(dT, format='sec')
        time = end.unix_tai - t0
        ims_data = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start, end)
        az = await client.select_time_series("lsst.sal.MTMount.azimuth","actualPosition", start, end)
        el = await client.select_time_series("lsst.sal.MTMount.elevation","actualPosition", start, end)
        try:
            zPosFinal = ims_data['zPosition'].values[-1]
            if zPosFinal > -0.002: # Test if mirror is raised
                #azFinal = az.values[-1][0]
                #if azFinal > 105.0 and azFinal < 185.0:
                els.append(el.values[-1][0])
                azs.append(az.values[-1][0])
                times.append(time)
                xPos.append(ims_data['xPosition'].values[-1] * 1.0E6)
                yPos.append(ims_data['yPosition'].values[-1] * 1.0E6)
                zPos.append(ims_data['zPosition'].values[-1] * 1.0E6)
                xRot.append(ims_data['xRotation'].values[-1] * 1.0E6)
                yRot.append(ims_data['yRotation'].values[-1] * 1.0E6)
                zRot.append(ims_data['zRotation'].values[-1] * 1.0E6)
        except:
            continue

    print(len(els))
    data[dayObs] = [els, azs, times, [xPos, yPos, zPos, xRot, yRot, zRot]]


In [ ]:
# This is checking the slew begin and end times.
# It looks correct
start = Time('2023-08-03T00:50:00', scale='utc')
end = Time('2023-08-03T01:00:00', scale='utc')
ims_data = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start, end)
az = await client.select_time_series("lsst.sal.MTMount.azimuth","actualPosition", start, end)
el = await client.select_time_series("lsst.sal.MTMount.elevation","actualPosition", start, end)


In [ ]:
fig, axs = plt.subplots(2,1)
plt.subplots_adjust(hspace=0.8)
el.plot(ax = axs[0])
axs[0].set_ylim(0,90)
axs[0].set_title("Elevation")
az.plot(ax = axs[1])
axs[1].set_ylim(-10,360)
axs[1].set_title('Azimuth')
for i, slew in enumerate(slews):
    if slew.begin.unix_tai > start.unix_tai and slew.begin.unix_tai < end.unix_tai \
    and slew.end.unix_tai > start.unix_tai and slew.end.unix_tai < end.unix_tai:
        print(i)
        axs[0].axvline(slew.begin.isot, ls='--', color='green')
        axs[0].axvline(slew.end.isot, ls='--', color='red')
        axs[1].axvline(slew.begin.isot, ls='--', color='green')
        axs[1].axvline(slew.end.isot, ls='--', color='red')
        

In [ ]:
# Checking if things have stabilized by 5 seconds after end of slew - yes!
slew = slews[160]
dT = 5.0
start = slew.end-TimeDelta(dT, format='sec')
end = slew.end+TimeDelta(dT, format='sec')
ims_data = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start, end)
az = await client.select_time_series("lsst.sal.MTMount.azimuth","actualPosition", start, end)
el = await client.select_time_series("lsst.sal.MTMount.elevation","actualPosition", start, end)


In [ ]:
%matplotlib inline
names = ['xPosition', 'yPosition', 'zPosition', 'xRotation', 'yRotation', 'zRotation']
limits = [6.0, 6.0, 1.0, 24.0, 24.0, 24.0]

fig, axs = plt.subplots(3,2,figsize=(8,10))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.suptitle(f"Final mirror position after a slew", fontsize=16)
plot_counter = 0
for i in range(3):
    for j in range(2):
        if plot_counter > 2:
            unit = 'microDeg'
        else:
            unit = 'um'
        for dayObs in data.keys():
            [els, azs, times, plots] = data[dayObs] 

            axs[i][j].set_title(names[plot_counter])
            axs[i][j].set_xlim(0, 90)
            axs[i][j].set_xlabel("Elevation (degrees)")
            axs[i][j].set_ylabel(f'IMS ({unit})')
            axs[i][j].scatter(els, plots[plot_counter], label=dayObs, c=times)
        plot_counter += 1
axs[0][1].legend()
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Final_Mirror_Position_Time_03Aug23.png")

## Repeat with attempting to log %max values

In [ ]:
blockTimes = ["2023-08-02T21:44:00", "2023-08-03T01:14:00", "2023-08-03T03:06:00", \
              "2023-08-03T05:01:00", "2023-08-03T06:00:00"]
azMaxLimits = [30.0, 40.0, 50.0, 20.0]
elMaxLimits = [30.0, 30.0, 30.0, 20.0]

In [ ]:
data = {}

for dayObs in [20230802]:
    els = []
    azs = []
    times = []
    elLimits = []
    azLimits = []
    xPos = []
    yPos = []
    zPos = []
    xRot = []
    yRot = []
    zRot = []

    events = eventMaker.getEvents(dayObs)
    slews = [e for e in events if e.type==TMAState.SLEWING]
    print(f'{dayObs}:found {len(slews)} slews ')
    if len(slews) < 1:
        continue
    t0 = slews[0].begin.unix_tai
    for slew in slews:

        dT = 5.0
        start = slew.end-TimeDelta(dT, format='sec')
        end = slew.end+TimeDelta(dT, format='sec')
        time = end.unix_tai - t0
        ims_data = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start, end)
        az = await client.select_time_series("lsst.sal.MTMount.azimuth","actualPosition", start, end)
        el = await client.select_time_series("lsst.sal.MTMount.elevation","actualPosition", start, end)
        try:
            zPosFinal = ims_data['zPosition'].values[-1]
            if zPosFinal > -0.002: # Test if mirror is raised
                for n, blockTime in enumerate(blockTimes[0:-1]):
                    startTime = Time(blockTime, scale='utc')
                    endTime = Time(blockTimes[n+1], scale='utc')
                    startTest = start.unix_tai - startTime.unix_tai
                    endTest = start.unix_tai - endTime.unix_tai
                    if startTest >0.0 and endTest < 0.0:
                        elLimits.append(elMaxLimits[n])
                        azLimits.append(azMaxLimits[n])
                        els.append(el.values[-1][0])
                        azs.append(az.values[-1][0])
                        times.append(time)
                        xPos.append(ims_data['xPosition'].values[-1] * 1.0E6)
                        yPos.append(ims_data['yPosition'].values[-1] * 1.0E6)
                        zPos.append(ims_data['zPosition'].values[-1] * 1.0E6)
                        xRot.append(ims_data['xRotation'].values[-1] * 1.0E6)
                        yRot.append(ims_data['yRotation'].values[-1] * 1.0E6)
                        zRot.append(ims_data['zRotation'].values[-1] * 1.0E6)
        except:
            continue

    print(len(els))
    data[dayObs] = [els, azs, times, elLimits, azLimits, [xPos, yPos, zPos, xRot, yRot, zRot]]


In [ ]:
%matplotlib inline
names = ['xPosition', 'yPosition', 'zPosition', 'xRotation', 'yRotation', 'zRotation']
limits = [6.0, 6.0, 1.0, 24.0, 24.0, 24.0]

fig, axs = plt.subplots(3,2,figsize=(8,10))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.suptitle(f"Final mirror position after a slew", fontsize=16)
plot_counter = 0
for i in range(3):
    for j in range(2):
        if plot_counter > 2:
            unit = 'microDeg'
        else:
            unit = 'um'
        for dayObs in data.keys():
            [els, azs, times, elLimits, azLimits, plots] = data[dayObs] 

            axs[i][j].set_title(names[plot_counter])
            axs[i][j].set_xlim(0, 90)
            axs[i][j].set_xlabel("Elevation (degrees)")
            axs[i][j].set_ylabel(f'IMS ({unit})')
            p = axs[i][j].scatter(els, plots[plot_counter], label=dayObs, c=azLimits)
            fig.colorbar(p,ax=axs[i][j],orientation='vertical',label='azLimit(%)')
        plot_counter += 1
        
axs[0][1].legend()

plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Final_Mirror_Position_AzLimits_03Aug23.png")

In [ ]:
data = {}

for dayObs in [20230802]:
    els = []
    azs = []
    times = []
    elLimits = []
    azLimits = []
    xPos = []
    yPos = []
    zPos = []
    xRot = []
    yRot = []
    zRot = []

    events = eventMaker.getEvents(dayObs)
    slews = [e for e in events if e.type==TMAState.SLEWING]
    print(f'{dayObs}:found {len(slews)} slews ')
    if len(slews) < 1:
        continue
    t0 = slews[0].begin.unix_tai
    for slew in slews:

        dT = 5.0
        start = slew.end-TimeDelta(dT, format='sec')
        end = slew.end+TimeDelta(dT, format='sec')
        time = end.unix_tai - t0
        ims_data = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start, end)
        az = await client.select_time_series("lsst.sal.MTMount.azimuth","actualPosition", start, end)
        el = await client.select_time_series("lsst.sal.MTMount.elevation","actualPosition", start, end)
        try:
            zPosFinal = ims_data['zPosition'].values[-1]
            if zPosFinal > -0.002: # Test if mirror is raised
                for n, blockTime in enumerate(blockTimes[0:-1]):
                    startTime = Time(blockTime, scale='utc')
                    endTime = Time(blockTimes[n+1], scale='utc')
                    startTest = start.unix_tai - startTime.unix_tai
                    endTest = start.unix_tai - endTime.unix_tai
                    if startTest >0.0 and endTest < 0.0:
                        elLimits.append(elMaxLimits[n])
                        azLimits.append(azMaxLimits[n])
                        els.append(el.values[-1][0])
                        azs.append(az.values[-1][0])
                        times.append(time)
                        xPos.append(ims_data['xPosition'].values[-1] * 1.0E6)
                        yPos.append(ims_data['yPosition'].values[-1] * 1.0E6)
                        zPos.append(ims_data['zPosition'].values[-1] * 1.0E6)
                        xRot.append(ims_data['xRotation'].values[-1] * 1.0E6)
                        yRot.append(ims_data['yRotation'].values[-1] * 1.0E6)
                        zRot.append(ims_data['zRotation'].values[-1] * 1.0E6)
        except:
            continue

    print(len(els))
    data[dayObs] = [els, azs, times, elLimits, azLimits, [xPos, yPos, zPos, xRot, yRot, zRot]]


In [ ]:
%matplotlib inline
names = ['xPosition', 'yPosition', 'zPosition', 'xRotation', 'yRotation', 'zRotation']
limits = [6.0, 6.0, 1.0, 24.0, 24.0, 24.0]

fig, axs = plt.subplots(2,3,figsize=(12,8))
plt.subplots_adjust(hspace=0.3, wspace=0.4)
plt.suptitle(f"Final mirror position after a slew", fontsize=16)
plot_counter = 0
for i in range(2):
    for j in range(3):
        if plot_counter > 2:
            unit = 'microDeg'
        else:
            unit = 'um'
        for dayObs in data.keys():
            [els, azs, times, elLimits, azLimits, plots] = data[dayObs] 

            axs[i][j].set_title(names[plot_counter])
            axs[i][j].set_xlim(0, 90)
            axs[i][j].set_xlabel("Elevation (degrees)")
            axs[i][j].set_ylabel(f'IMS ({unit})')
            for m, azLimit in enumerate(azMaxLimits):
                theseEls = []
                theseData = []
                for n in range(len(els)):
                    if abs(azLimits[n] - azLimit) < 0.1:
                        theseEls.append(els[n])
                        theseData.append(plots[plot_counter][n])
                
                axs[i][j].scatter(theseEls, theseData, label=azLimit)
                fit = np.polyfit(theseEls, theseData, 2)
                #print(azLimit, fit)
                xplot = np.linspace(0.0, 90.0, 1000)
                yplot = np.polyval(fit, xplot)
                axs[i][j].plot(xplot, yplot, ls='--')

                axs[i][j].legend()
        plot_counter += 1
        


plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Final_Mirror_Position_AzLimits_3_03Aug23.png")

In [ ]:
data = {}

for dayObs in [20230802]:
    els = []
    azs = []
    times = []
    temps = []
    elLimits = []
    azLimits = []
    xPos = []
    yPos = []
    zPos = []
    xRot = []
    yRot = []
    zRot = []

    events = eventMaker.getEvents(dayObs)
    slews = [e for e in events if e.type==TMAState.SLEWING]
    print(f'{dayObs}:found {len(slews)} slews ')
    if len(slews) < 1:
        continue
    t0 = slews[0].begin.unix_tai
    for slew in slews:

        dT = 5.0
        start = slew.end-TimeDelta(dT, format='sec')
        end = slew.end+TimeDelta(dT, format='sec')
        time = end.unix_tai - t0
        ims_data = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start, end)
        az = await client.select_time_series("lsst.sal.MTMount.azimuth","actualPosition", start, end)
        el = await client.select_time_series("lsst.sal.MTMount.elevation","actualPosition", start, end)
        temp = await client.select_time_series("lsst.sal.MTM1M3TS.glycolLoopTemperature", "insideCellTemperature1", start, end)
        try:
            zPosFinal = ims_data['zPosition'].values[-1]
        except:
            continue

        if zPosFinal > -0.002: # Test if mirror is raised
            for n, blockTime in enumerate(blockTimes[0:-1]):
                startTime = Time(blockTime, scale='utc')
                endTime = Time(blockTimes[n+1], scale='utc')
                startTest = start.unix_tai - startTime.unix_tai
                endTest = start.unix_tai - endTime.unix_tai
                if startTest >0.0 and endTest < 0.0:
                    elLimits.append(elMaxLimits[n])
                    azLimits.append(azMaxLimits[n])
                    els.append(el.values[-1][0])
                    azs.append(az.values[-1][0])
                    times.append(time)
                    temps.append(np.median(temp['insideCellTemperature1'].values))
                    xPos.append(ims_data['xPosition'].values[-1] * 1.0E6)
                    yPos.append(ims_data['yPosition'].values[-1] * 1.0E6)
                    zPos.append(ims_data['zPosition'].values[-1] * 1.0E6)
                    xRot.append(ims_data['xRotation'].values[-1] * 1.0E6)
                    yRot.append(ims_data['yRotation'].values[-1] * 1.0E6)
                    zRot.append(ims_data['zRotation'].values[-1] * 1.0E6)

    print(len(els))
    data[dayObs] = [els, azs, times, temps, elLimits, azLimits, [xPos, yPos, zPos, xRot, yRot, zRot]]


In [ ]:
temps

In [ ]:
%matplotlib inline
names = ['xPosition', 'yPosition', 'zPosition', 'xRotation', 'yRotation', 'zRotation']
limits = [6.0, 6.0, 1.0, 24.0, 24.0, 24.0]

fig, axs = plt.subplots(2,3,figsize=(12,8))
plt.subplots_adjust(hspace=0.3, wspace=0.4)
plt.suptitle(f"Final mirror position after a slew", fontsize=16)
plot_counter = 2#0
theseEls = []
theseData = []
theseTemps = []
theseTimes = []
for i in [0]:#range(2):
    for j in [2]:#range(3):
        if plot_counter > 2:
            unit = 'microDeg'
        else:
            unit = 'um'
        for dayObs in data.keys():
            [els, azs, times, temps, elLimits, azLimits, plots] = data[dayObs] 

            axs[i][j].set_title(names[plot_counter])
            axs[i][j].set_xlim(0, 90)
            axs[i][j].set_xlabel("Elevation (degrees)")
            axs[i][j].set_ylabel(f'IMS ({unit})')
            for m, azLimit in enumerate(azMaxLimits):
                for n in range(len(els)):
                    if abs(azLimits[n] - azLimit) < 0.1:
                        theseTimes.append(times[n])
                        theseEls.append(els[n])
                        theseData.append(plots[plot_counter][n])
                        theseTemps.append(temps[n])
                
                axs[i][j].scatter(theseEls, theseData, label=azLimit)
                fit = np.polyfit(theseEls, theseData, 2)
                #print(azLimit, fit)
                xplot = np.linspace(0.0, 90.0, 1000)
                yplot = np.polyval(fit, xplot)
                axs[i][j].plot(xplot, yplot, ls='--')

                axs[i][j].legend()
        plot_counter += 1
        


plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Final_Mirror_Position_AzLimits_3_03Aug23.png")

In [ ]:
print(len(theseEls), len(theseData), len(theseTemps), len(theseTimes))
print(theseTimes[0])

In [ ]:
plt.scatter(theseTimes, theseTemps)

In [ ]:
from scipy.optimize import minimize

def func1(params, args):
    [const, El1, El2] = params
    [els, data, temps] = args
    diff = data - (const + El1 * els + El2 * els * els)
    return np.sum(diff * diff)

def func2(params, args):
    [const, El1, El2, T1] = params
    [els, data, temps] = args
    diff = data - (const + El1 * els + El2 * els * els + T1 * temps)
    return np.sum(diff * diff)

In [ ]:
args = [np.array(theseEls), np.array(theseData), np.array(theseTemps)]
param0 = [-700, 1.7, -.01, -60.0]
Tparams = minimize(func2, param0, args=args, method='Powell').x
param0 = [-700, 1.7, -.01]
NoTparams = minimize(func1, param0, args=args, method='Powell').x

In [ ]:
NoTparams

In [ ]:
fig, axs = plt.subplots(2,2,figsize = (10,10))
plt.subplots_adjust(wspace = 0.5)
plt.suptitle("Mirror Z Position during slews 20230803", fontsize = 24)

axs[0][0].set_title("No temperature compensation")
axs[0][0].scatter(theseEls, theseData)
axs[0][0].set_xlabel("Elevation (degrees)")
axs[0][0].set_ylabel("Mirror zPosition (microns)")
xs = np.linspace(20,90,1000)
ys = NoTparams[0] + NoTparams[1] * xs + NoTparams[2] * xs * xs
axs[0][0].plot(xs, ys, ls = '--', color='red')

axs[0][1].set_title("With temperature compensation")
axs[0][1].scatter(theseEls, theseData - Tparams[3] * np.array(theseTemps))
axs[0][1].set_xlabel("Elevation (degrees)")
axs[0][1].set_ylabel("Temp adjusted Mirror zPosition (microns)")
xs = np.linspace(20,90,1000)
ys = Tparams[0] + Tparams[1] * xs + Tparams[2] * xs * xs
axs[0][1].plot(xs, ys, ls = '--', color='red')

axs[1][0].set_title("Temperature vs Time")
axs[1][0].scatter(theseTimes, theseTemps)
axs[1][0].set_xlabel("Time(seconds)")
axs[1][0].set_ylabel("lsst.sal.MTM1M3TS.glycolLoopTemperature.insideCellTemperature1 (C)", fontsize=8)

axs[1][1].axis('off')

axs[0][0].plot([60,70],[-1440,-1440], ls='--', color='black')
axs[0][0].plot([60,70],[-1442,-1442], ls='--', color='black')
axs[0][0].text(71,-1442, "Spec limits", color='black')
axs[0][1].plot([60,70],[-670,-670], ls='--', color='black')
axs[0][1].plot([60,70],[-672,-672], ls='--', color='black')
axs[0][1].text(71,-672, "Spec limits", color='black')


plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Mirror_Position_Temperature_03Aug23.png")